## 初始设置

In [57]:
import argparse 
parser = argparse.ArgumentParser(description='pytorch version of GraphSAGE')
parser.add_argument('--dataSet', type=str, default='cora')
parser.add_argument('--agg_func', type=str, default='MEAN')
parser.add_argument('--epochs', type=int, default=50)
parser.add_argument('--b_sz', type=int, default=20)
parser.add_argument('--seed', type=int, default=824)
parser.add_argument('--cuda', action='store_true',
					help='use CUDA')
parser.add_argument('--gcn', action='store_true')
parser.add_argument('--learn_method', type=str, default='sup')
parser.add_argument('--unsup_loss', type=str, default='normal')
parser.add_argument('--max_vali_f1', type=float, default=0)
parser.add_argument('--name', type=str, default='debug')
parser.add_argument('--config', type=str, default='./src/experiments.conf')
args, _ = parser.parse_known_args()

In [58]:
import torch
if torch.cuda.is_available():
	if not args.cuda:
		print("WARNING: You have a CUDA device, so you should probably run with --cuda")
	else:
		device_id = torch.cuda.current_device()
		print('using device', device_id, torch.cuda.get_device_name(device_id))

device = torch.device("cuda" if args.cuda else "cpu")
print('DEVICE:', device)

DEVICE: cpu


## 加载config文件

In [59]:
import pyhocon
import random
import numpy as np

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)

config = pyhocon.ConfigFactory.parse_file(args.config)

## 定义DataCenter类

In [60]:
from collections import defaultdict
class DataCenter(object):
	"""docstring for DataCenter"""
	def __init__(self, config):
		super(DataCenter, self).__init__()
		self.config = config
		
	def load_dataSet(self, dataSet='cora'):
		##设置读取路径
		if dataSet == 'cora':
			cora_content_file = config['file_path.cora_content']
			cora_cite_file = config['file_path.cora_cite']
			
			feat_data = []# 节点特征矩阵
			labels = [] # 标签列表
			node_map = {} # 节点索引
			label_map = {} # 标签索引
			with open(cora_content_file) as fp:
				for i,line in enumerate(fp):
					##分割行
					info = line.strip().split()
					##遍历出每个列中的第二个到倒数第二个，从而获得节点特征表示		
					feat_data.append([float(x) for x in info[1:-1]])
					##获取节点编号并且将其转化为节点索引
					node_map[info[0]] = i
					##如果标签列表中的最后一个位置没有标签索引
					if not info[-1] in label_map:
					##将他的标签索引设定为它所在位置索引
						label_map[info[-1]] = len(label_map)
					##将其进行标签索引代入标签列表
					labels.append(label_map[info[-1]])
					##将节点特征列表转化为数组形式
			feat_data = np.asarray(feat_data)
					##将标签列表转化为数组形式 
			labels = np.asarray(labels, dtype=np.int64)
			
			##将元组转化为列表	
			adj_lists = defaultdict(set)
			with open(cora_cite_file) as fp:
				for i,line in enumerate(fp):
					## 将边转化为两个节点
					info = line.strip().split()
					##判断是否转化后是两个节点
					assert len(info) == 2
					##将起始节点赋值到paper1中将并将节点编号转化为索引
					paper1 = node_map[info[0]]
					##将截止节点复制到paper2中将并将节点编号转化为索引
					paper2 = node_map[info[1]]
					##形成节点和邻接节点连接的字典
					adj_lists[paper1].add(paper2)
					adj_lists[paper2].add(paper1)
					##判断节点特征个数应该等于标签个数应该等于节点和邻接节点连接的字典
			assert len(feat_data) == len(labels) == len(adj_lists)
			test_indexs, val_indexs, train_indexs = self._split_data(feat_data.shape[0])

			setattr(self, dataSet+'_test', test_indexs)
			setattr(self, dataSet+'_val', val_indexs)
			setattr(self, dataSet+'_train', train_indexs)

			setattr(self, dataSet+'_feats', feat_data)
			setattr(self, dataSet+'_labels', labels)
			setattr(self, dataSet+'_adj_lists', adj_lists)
			
##划分数据集、验证集和训练集	
	def _split_data(self, num_nodes, test_split = 3, val_split = 6):
		rand_indices = np.random.permutation(num_nodes)

		test_size = num_nodes // test_split
		val_size = num_nodes // val_split
		train_size = num_nodes - (test_size + val_size)

		test_indexs = rand_indices[:test_size]
		val_indexs = rand_indices[test_size:(test_size+val_size)]
		train_indexs = rand_indices[(test_size+val_size):]
		print(f'训练节点有{len(train_indexs)}个')
		print(f'测试节点有{len(test_indexs)}个')
		print(f'验证节点有{len(val_indexs)}个')
		
		return test_indexs, val_indexs, train_indexs

In [61]:
ds = args.dataSet
dataCenter = DataCenter(config)
dataCenter.load_dataSet(ds)
features = torch.FloatTensor(getattr(dataCenter, ds+'_feats')).to(device)

训练节点有1355个
测试节点有902个
验证节点有451个


## sagelayer的定义

In [62]:
import torch.nn as nn
import torch.nn.functional as F
class SageLayer(nn.Module):
	"""
	Encodes a node's using 'convolutional' GraphSage approach
	"""
	def __init__(self, input_size, out_size, gcn=False): 
		super(SageLayer, self).__init__()
	##输入维度
		self.input_size = input_size
	#输出维度
		self.out_size = out_size
	##gcn
		self.gcn = gcn
	##可学习的权重参数，如果使用gcn就是输入输入维度，如果是不使用gcn那就是输入输入维度的两倍
		self.weight = nn.Parameter(torch.FloatTensor(out_size, self.input_size if self.gcn else 2 * self.input_size))
	##初始化参数
		self.init_params()

	def init_params(self):
		for param in self.parameters():
			nn.init.xavier_uniform_(param)

	def forward(self, self_feats, aggregate_feats, neighs=None):
		"""
		Generates embeddings for a batch of nodes.

		nodes	 -- list of nodes
		"""
		if not self.gcn:
			##结合节点表征和聚合后的特征
			combined = torch.cat([self_feats, aggregate_feats], dim=1)
		else:
			combined = aggregate_feats
			##将其与一个可学习的权重参数并且进行非线性
		combined = F.relu(self.weight.mm(combined.t())).t()
		return combined


## graphsage定义

In [63]:
class GraphSage(nn.Module):
	"""docstring for GraphSage"""
	def __init__(self, num_layers, input_size, out_size, raw_features, adj_lists, device, gcn=False, agg_func='MEAN'):
		super(GraphSage, self).__init__()
	## 设定输出维度
		self.input_size = input_size
	## 规定输入维度
		self.out_size = out_size
	## 规定层数
		self.num_layers = num_layers
	## 规定gcn
		self.gcn = gcn
	## 规定cpu or gpu
		self.device = device
	## 定义聚合函数
		self.agg_func = agg_func
	## 点特征
		self.raw_features = raw_features
	## 边的连接
		self.adj_lists = adj_lists

		for index in range(1, num_layers+1):
	##如果是第一层，那么该层的输出维度等于input size，如果是第二层，那么输出维度等于output_size
			layer_size = out_size if index != 1 else input_size
			setattr(self, 'sage_layer'+str(index), SageLayer(layer_size, out_size, gcn=self.gcn))

	def forward(self, nodes_batch):
##采样
		##导入训练的batch节点
		lower_layer_nodes = list(nodes_batch)

		##聚合节点的层
		nodes_batch_layers = [(lower_layer_nodes,)]

		# 每一层的graph sage
		for i in range(self.num_layers):
		##获得邻接节点
			lower_samp_neighs, lower_layer_nodes_dict, lower_layer_nodes= self._get_unique_neighs_list(lower_layer_nodes)
		##形成layer2（最外层节点），layer1（中间层节点），layer center（中心节点）
			nodes_batch_layers.insert(0, (lower_layer_nodes, lower_samp_neighs, lower_layer_nodes_dict))

		assert len(nodes_batch_layers) == self.num_layers + 1

##聚合
		##将节点特征赋予到变量pre_hidden_embs
		pre_hidden_embs = self.raw_features
		##nodes_batch_layers=[layer2,layer1,layer_center]
		for index in range(1, self.num_layers+1):
		##以layer1节点作为中心节点
			nb = nodes_batch_layers[index][0]
		##取中心节点对应的上层（layer2）的邻居节点
			pre_neighs = nodes_batch_layers[index-1]

		##聚合函数进行聚合得到中心节点特征
			aggregate_feats = self.aggregate(nb, pre_hidden_embs, pre_neighs)
			sage_layer = getattr(self, 'sage_layer'+str(index))

		##如果层数大于1
			if index > 1:
				nb = self._nodes_map(nb, pre_hidden_embs, pre_neighs)
		##利用中心节点表征和聚合后的中心节点特征进行拼接然后连一个可学习的权重参数
			cur_hidden_embs = sage_layer(self_feats=pre_hidden_embs[nb],
										aggregate_feats=aggregate_feats)
			pre_hidden_embs = cur_hidden_embs

		return pre_hidden_embs

	def _nodes_map(self, nodes, hidden_embs, neighs):
		layer_nodes, samp_neighs, layer_nodes_dict = neighs
		assert len(samp_neighs) == len(nodes)
		index = [layer_nodes_dict[x] for x in nodes]
		return index

	##获得节点的邻居
	def _get_unique_neighs_list(self, nodes, num_sample=10):
		_set = set
		##获取每个节点的邻居节点
		to_neighs = [self.adj_lists[int(node)] for node in nodes]

		##进行采样
		if not num_sample is None:
			_sample = random.sample
		##如果邻居节点个数大于等于十个，则随机取十个，如果邻居节点个数小于十个则全取
			samp_neighs = [_set(_sample(to_neigh, num_sample)) if len(to_neigh) >= num_sample else to_neigh for to_neigh in to_neighs]

		else:
			samp_neighs = to_neighs
		##将采样的邻居节点和本身的中心节点结合得到节点所属邻居的集合
		samp_neighs = [samp_neigh | set([nodes[i]]) for i, samp_neigh in enumerate(samp_neighs)]

		##进行去重操作
		_unique_nodes_list = list(set.union(*samp_neighs))

		##进行重新排列
		i = list(range(len(_unique_nodes_list)))

		##对字典进行重新编号
		unique_nodes = dict(list(zip(_unique_nodes_list, i)))

		##返回所有节点的邻居，返回所有节点的编号，返回所有节点的列表
		return samp_neighs, unique_nodes, _unique_nodes_list


	def aggregate(self, nodes, pre_hidden_embs, pre_neighs, num_sample=10):

		##取出最外层邻居节点（layer2）
		unique_nodes_list, samp_neighs, unique_nodes = pre_neighs
		assert len(nodes) == len(samp_neighs)

		##判断邻居节点是否包含了中心节点
		indicator = [(nodes[i] in samp_neighs[i]) for i in range(len(samp_neighs))]
		assert (False not in indicator)

		##将邻居节点包含中心节点的部分去掉
		if not self.gcn:
			samp_neighs = [(samp_neighs[i]-set([nodes[i]])) for i in range(len(samp_neighs))]

		##判断如果涉及到所有中心节点，保留原先矩阵，如果不涉及所有中心，保留部分矩阵
		if len(pre_hidden_embs) == len(unique_nodes):
			embed_matrix = pre_hidden_embs
		else:
			embed_matrix = pre_hidden_embs[torch.LongTensor(unique_nodes_list)]

		##以邻接节点为行，以中心节点为列构建邻接矩阵
		mask = torch.zeros(len(samp_neighs), len(unique_nodes))
		column_indices = [unique_nodes[n] for samp_neigh in samp_neighs for n in samp_neigh]
		row_indices = [i for i in range(len(samp_neighs)) for j in range(len(samp_neighs[i]))]

		##将其有链接的地方记为1
		mask[row_indices, column_indices] = 1



		##选择平均的方式进行聚合
		if self.agg_func == 'MEAN':
		##按行求和得到每个中心节点的连接的邻居节点的个数
			num_neigh = mask.sum(1, keepdim=True)
		##按行进行归一化操作
			mask = mask.div(num_neigh).to(embed_matrix.device)
		##矩阵相乘，相当于聚合周围临界信息并求和
			aggregate_feats = mask.mm(embed_matrix)

		elif self.agg_func == 'MAX':
			# print(mask)
			indexs = [x.nonzero() for x in mask==1]
			aggregate_feats = []
			# self.dc.logger.info('5')
			for feat in [embed_matrix[x.squeeze()] for x in indexs]:
				if len(feat.size()) == 1:
					aggregate_feats.append(feat.view(1, -1))
				else:
					aggregate_feats.append(torch.max(feat,0)[0].view(1, -1))
			aggregate_feats = torch.cat(aggregate_feats, 0)

		# self.dc.logger.info('6')
		
		return aggregate_feats

In [64]:
graphSage = GraphSage(config['setting.num_layers'], features.size(1), config['setting.hidden_emb_size'], features, getattr(dataCenter, ds+'_adj_lists'), device, gcn=args.gcn, agg_func=args.agg_func)
graphSage.to(device)

GraphSage(
  (sage_layer1): SageLayer()
  (sage_layer2): SageLayer()
)

In [65]:
##定义label的数量
num_labels = len(set(getattr(dataCenter, ds+'_labels')))

## 定义分类器

In [66]:
class Classification(nn.Module):

	def __init__(self, emb_size, num_classes):
		super(Classification, self).__init__()

		#self.weight = nn.Parameter(torch.FloatTensor(emb_size, num_classes))
		self.layer = nn.Sequential(
								nn.Linear(emb_size, num_classes)	  
								#nn.ReLU()
							)
		self.init_params()

	def init_params(self):
		for param in self.parameters():
			if len(param.size()) == 2:
				nn.init.xavier_uniform_(param)

	def forward(self, embeds):
		logists = torch.log_softmax(self.layer(embeds), 1)
		return logists

In [67]:
classification = Classification(config['setting.hidden_emb_size'], num_labels)
classification.to(device)

Classification(
  (layer): Sequential(
    (0): Linear(in_features=128, out_features=7, bias=True)
  )
)

## 无监督loss

In [68]:
class UnsupervisedLoss(object):
	"""docstring for UnsupervisedLoss"""
	def __init__(self, adj_lists, train_nodes, device):
		super(UnsupervisedLoss, self).__init__()
		self.Q = 10
		self.N_WALKS = 6
		self.WALK_LEN = 1
		self.N_WALK_LEN = 5
		self.MARGIN = 3
		self.adj_lists = adj_lists
		self.train_nodes = train_nodes
		self.device = device

		self.target_nodes = None
		self.positive_pairs = []
		self.negtive_pairs = []
		self.node_positive_pairs = {}
		self.node_negtive_pairs = {}
		self.unique_nodes_batch = []

	def get_loss_sage(self, embeddings, nodes):
	## 确保输入的嵌入(embeddings)和唯一节点批次(unique_nodes_batch)的长度相同
		assert len(embeddings) == len(self.unique_nodes_batch)
		assert False not in [nodes[i]==self.unique_nodes_batch[i] for i in range(len(nodes))]
	##建立节点和嵌入列表的索引的映射关系
		node2index = {n:i for i,n in enumerate(self.unique_nodes_batch)}
	
	##对每个节点，计算它的正负样本对的分数
		nodes_score = []
		assert len(self.node_positive_pairs) == len(self.node_negtive_pairs)
		for node in self.node_positive_pairs:
			pps = self.node_positive_pairs[node]
			nps = self.node_negtive_pairs[node]
			if len(pps) == 0 or len(nps) == 0:
				continue
		## 负样本对的损失，使用余弦相似度和对数sigmoid函数
			# Q * Exception(negative score)
			indexs = [list(x) for x in zip(*nps)]
			node_indexs = [node2index[x] for x in indexs[0]]
			neighb_indexs = [node2index[x] for x in indexs[1]]
			neg_score = F.cosine_similarity(embeddings[node_indexs], embeddings[neighb_indexs])
			neg_score = self.Q*torch.mean(torch.log(torch.sigmoid(-neg_score)), 0)
			#print(neg_score)
		
		## 正样本对的损失，同样使用余弦相似度和对数sigmoid函数。
			# multiple positive score
			indexs = [list(x) for x in zip(*pps)]
			node_indexs = [node2index[x] for x in indexs[0]]
			neighb_indexs = [node2index[x] for x in indexs[1]]
			pos_score = F.cosine_similarity(embeddings[node_indexs], embeddings[neighb_indexs])
			pos_score = torch.log(torch.sigmoid(pos_score))
			#print(pos_score)

			nodes_score.append(torch.mean(- pos_score - neg_score).view(1,-1))
				
		loss = torch.mean(torch.cat(nodes_score, 0))
		
		return loss
	
##边缘损失（margin loss）
	##这个方法与get_loss_sage类似，但在计算损失时使用了边缘损失
	def get_loss_margin(self, embeddings, nodes):
		assert len(embeddings) == len(self.unique_nodes_batch)
		assert False not in [nodes[i]==self.unique_nodes_batch[i] for i in range(len(nodes))]
		node2index = {n:i for i,n in enumerate(self.unique_nodes_batch)}

		nodes_score = []
		assert len(self.node_positive_pairs) == len(self.node_negtive_pairs)
		for node in self.node_positive_pairs:
			pps = self.node_positive_pairs[node]
			nps = self.node_negtive_pairs[node]
			if len(pps) == 0 or len(nps) == 0:
				continue
		##正样本对的最小得分
			indexs = [list(x) for x in zip(*pps)]
			node_indexs = [node2index[x] for x in indexs[0]]
			neighb_indexs = [node2index[x] for x in indexs[1]]
			pos_score = F.cosine_similarity(embeddings[node_indexs], embeddings[neighb_indexs])
			pos_score, _ = torch.min(torch.log(torch.sigmoid(pos_score)), 0)
		## 负样本对的最大得分
			indexs = [list(x) for x in zip(*nps)]
			node_indexs = [node2index[x] for x in indexs[0]]
			neighb_indexs = [node2index[x] for x in indexs[1]]
			neg_score = F.cosine_similarity(embeddings[node_indexs], embeddings[neighb_indexs])
			neg_score, _ = torch.max(torch.log(torch.sigmoid(neg_score)), 0)
			
		##计算它们之间的差异，加上一个边缘值
			nodes_score.append(torch.max(torch.tensor(0.0).to(self.device), neg_score-pos_score+self.MARGIN).view(1,-1))
			# nodes_score.append((-pos_score - neg_score).view(1,-1))

		loss = torch.mean(torch.cat(nodes_score, 0),0)

		# loss = -torch.log(torch.sigmoid(pos_score))-4*torch.log(torch.sigmoid(-neg_score))
		
		return loss


	def extend_nodes(self, nodes, num_neg=6):
	##清空当前的正负样本对和它们的映射
		self.positive_pairs = []
		self.node_positive_pairs = {}
		self.negtive_pairs = []
		self.node_negtive_pairs = {}
	##设置目标节点为传入的节点
		self.target_nodes = nodes
	##方法来生成正负样本对。
		self.get_positive_nodes(nodes)
		# print(self.positive_pairs)
		self.get_negtive_nodes(nodes, num_neg)
		# print(self.negtive_pairs)
	##所有正负样本对中出现的唯一节点
		self.unique_nodes_batch = list(set([i for x in self.positive_pairs for i in x]) | set([i for x in self.negtive_pairs for i in x]))
		assert set(self.target_nodes) < set(self.unique_nodes_batch)
		return self.unique_nodes_batch
	##正样本生成——_run_random_walks方法生成正样本对
	def get_positive_nodes(self, nodes):
		return self._run_random_walks(nodes)
	## 每个节点生成指定数量的负样本。这些负样本是从不是节点邻居的节点中随机选取的
	def get_negtive_nodes(self, nodes, num_neg):
		for node in nodes:
			neighbors = set([node])
			frontier = set([node])
			for i in range(self.N_WALK_LEN):
				current = set()
				for outer in frontier:
					current |= self.adj_lists[int(outer)]
				frontier = current - neighbors
				neighbors |= current
			far_nodes = set(self.train_nodes) - neighbors
			neg_samples = random.sample(far_nodes, num_neg) if num_neg < len(far_nodes) else far_nodes
			self.negtive_pairs.extend([(node, neg_node) for neg_node in neg_samples])
			self.node_negtive_pairs[node] = [(node, neg_node) for neg_node in neg_samples]
		return self.negtive_pairs
	##随机游走函数
	def _run_random_walks(self, nodes):
		for node in nodes:
			if len(self.adj_lists[int(node)]) == 0:
				continue
			cur_pairs = []
		##对于传入的每个节点，进行指定次数的随机游走。
			for i in range(self.N_WALKS):
				curr_node = node
				##对于每次随机游走进行指定长度并且每次随机游走都选择邻居节点作为下一个节点
				for j in range(self.WALK_LEN):
					neighs = self.adj_lists[int(curr_node)]
					next_node = random.choice(list(neighs))
					# self co-occurrences are useless
					##如果选中的邻居节点不是原始节点且在训练节点集中，将其作为正样本对添加到列表中
					if next_node != node and next_node in self.train_nodes:
						self.positive_pairs.append((node,next_node))
						cur_pairs.append((node,next_node))
					curr_node = next_node

			self.node_positive_pairs[node] = cur_pairs
		return self.positive_pairs
		

In [69]:
unsupervised_loss = UnsupervisedLoss(getattr(dataCenter, ds+'_adj_lists'), getattr(dataCenter, ds+'_train'), device)

## 正式训练

In [70]:
import sys
def apply_model(dataCenter, ds, graphSage, classification, unsupervised_loss, b_sz, unsup_loss, device, learn_method):
	##取出训练集、测试集、验证集的节点和标签
	test_nodes = getattr(dataCenter, ds+'_test')
	val_nodes = getattr(dataCenter, ds+'_val')
	train_nodes = getattr(dataCenter, ds+'_train')
	labels = getattr(dataCenter, ds+'_labels')

	##无监督学习的定义
	if unsup_loss == 'margin':
		num_neg = 6
	elif unsup_loss == 'normal':
		num_neg = 100
	else:
		print("unsup_loss can be only 'margin' or 'normal'.")
		sys.exit(1)

	##打乱训练集节点
	train_nodes = shuffle(train_nodes)
	##模型是由两个部分graphsage和classification组成的
	models = [graphSage, classification]
	params = []
	##遍历模型
	for model in models:
	##遍历模型的参数
		for param in model.parameters():
	##设置可训练的模式
			if param.requires_grad:
				params.append(param)

	##定义optimizer
	optimizer = torch.optim.SGD(params, lr=0.7)
	##梯度归0
	optimizer.zero_grad()
	for model in models:
		model.zero_grad()

	#得到训练批次
	batches = math.ceil(len(train_nodes) / b_sz)

	visited_nodes = set()

	##遍历每个batch
	for index in range(batches):
	##得到batch的节点数量
		nodes_batch = train_nodes[index*b_sz:(index+1)*b_sz]

	##无监督学习下的拓采样，对有监督学习并没有影响
		nodes_batch = np.asarray(list(unsupervised_loss.extend_nodes(nodes_batch, num_neg=num_neg)))
		visited_nodes |= set(nodes_batch)

	##得到该批次下的节点的标签
		labels_batch = labels[nodes_batch]

	##将批次节点输入得到节点嵌入向量
		embs_batch = graphSage(nodes_batch)

		if learn_method == 'sup':
			# superivsed learning
			#用softmax的全连接进行最后的输出分类
			logists = classification(embs_batch)
			loss_sup = -torch.sum(logists[range(logists.size(0)), labels_batch], 0)
			loss_sup /= len(nodes_batch)
			loss = loss_sup
		elif learn_method == 'plus_unsup':
			# superivsed learning
			logists = classification(embs_batch)
			loss_sup = -torch.sum(logists[range(logists.size(0)), labels_batch], 0)
			loss_sup /= len(nodes_batch)
			# unsuperivsed learning
			if unsup_loss == 'margin':
				loss_net = unsupervised_loss.get_loss_margin(embs_batch, nodes_batch)
			elif unsup_loss == 'normal':
				loss_net = unsupervised_loss.get_loss_sage(embs_batch, nodes_batch)
			loss = loss_sup + loss_net
		else:
			if unsup_loss == 'margin':
				loss_net = unsupervised_loss.get_loss_margin(embs_batch, nodes_batch)
			elif unsup_loss == 'normal':
				loss_net = unsupervised_loss.get_loss_sage(embs_batch, nodes_batch)
			loss = loss_net

		print('Step [{}/{}], Loss: {:.4f}, Dealed Nodes [{}/{}] '.format(index+1, batches, loss.item(), len(visited_nodes), len(train_nodes)))
		loss.backward()
		for model in models:
			nn.utils.clip_grad_norm_(model.parameters(), 5)
		optimizer.step()

		optimizer.zero_grad()
		for model in models:
			model.zero_grad()

	return graphSage, classification

In [71]:
import math
##从已经训练好的GraphSAGE模型中获取节点嵌入（embeddings）
def get_gnn_embeddings(gnn_model, dataCenter, ds):
    print('Loading embeddings from trained GraphSAGE model.')
	##这里创建一个全零矩阵，用于存储节点嵌入，矩阵的大小是节点数量乘以节点嵌入的维度。
    features = np.zeros((len(getattr(dataCenter, ds+'_labels')), gnn_model.out_size))
    ##这里创建一个包含所有节点索引的列表
    nodes = np.arange(len(getattr(dataCenter, ds+'_labels'))).tolist()
    b_sz = 500
    batches = math.ceil(len(nodes) / b_sz)
    embs = []
    
    for index in range(batches):
		##会从nodes列表中选取一部分节点
        nodes_batch = nodes[index*b_sz:(index+1)*b_sz]
		##使用gnn_model模型获取这些节点的嵌入
        embs_batch = gnn_model(nodes_batch)
		##然后，它会确保获取的嵌入数量和节点数量相同
        assert len(embs_batch) == len(nodes_batch)
		##将嵌入添加到embs列表中。
        embs.append(embs_batch)
        # if ((index+1)*b_sz) % 10000 == 0:
        #     print(f'Dealed Nodes [{(index+1)*b_sz}/{len(nodes)}]')

    assert len(embs) == batches
    embs = torch.cat(embs, 0)
    assert len(embs) == len(nodes)
    print('Embeddings loaded.')
    return embs.detach()

In [72]:
from sklearn.metrics import f1_score
def evaluate(dataCenter, ds, graphSage, classification, device, max_vali_f1, name, cur_epoch):
	test_nodes = getattr(dataCenter, ds+'_test')
	val_nodes = getattr(dataCenter, ds+'_val')
	labels = getattr(dataCenter, ds+'_labels')

	models = [graphSage, classification]

	params = []
	for model in models:
		for param in model.parameters():
			if param.requires_grad:
				param.requires_grad = False
				params.append(param)

	embs = graphSage(val_nodes)
	logists = classification(embs)
	_, predicts = torch.max(logists, 1)
	labels_val = labels[val_nodes]
	assert len(labels_val) == len(predicts)
	comps = zip(labels_val, predicts.data)

	vali_f1 = f1_score(labels_val, predicts.cpu().data, average="micro")
	print("Validation F1:", vali_f1)

	if vali_f1 > max_vali_f1:
		max_vali_f1 = vali_f1
		embs = graphSage(test_nodes)
		logists = classification(embs)
		_, predicts = torch.max(logists, 1)
		labels_test = labels[test_nodes]
		assert len(labels_test) == len(predicts)
		comps = zip(labels_test, predicts.data)

		test_f1 = f1_score(labels_test, predicts.cpu().data, average="micro")
		print("Test F1:", test_f1)

		for param in params:
			param.requires_grad = True

		torch.save(models, 'models/model_best_{}_ep{}_{:.4f}.torch'.format(name, cur_epoch, test_f1))

	for param in params:
		param.requires_grad = True

	return max_vali_f1

In [73]:
from sklearn.utils import shuffle
## 分类器训练
def train_classification(dataCenter, graphSage, classification, ds, device, max_vali_f1, name, epochs=800):
	print('Training Classification ...')
	c_optimizer = torch.optim.SGD(classification.parameters(), lr=0.5)
	# train classification, detached from the current graph
	#classification.init_params()
	b_sz = 50
	train_nodes = getattr(dataCenter, ds+'_train')
	labels = getattr(dataCenter, ds+'_labels')
	features = get_gnn_embeddings(graphSage, dataCenter, ds)
	for epoch in range(epochs):
		train_nodes = shuffle(train_nodes)
		batches = math.ceil(len(train_nodes) / b_sz)
		visited_nodes = set()
		for index in range(batches):
			nodes_batch = train_nodes[index*b_sz:(index+1)*b_sz]
			visited_nodes |= set(nodes_batch)
			labels_batch = labels[nodes_batch]
			embs_batch = features[nodes_batch]

			logists = classification(embs_batch)
			loss = -torch.sum(logists[range(logists.size(0)), labels_batch], 0)
			loss /= len(nodes_batch)
			# print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Dealed Nodes [{}/{}] '.format(epoch+1, epochs, index, batches, loss.item(), len(visited_nodes), len(train_nodes)))

			loss.backward()
			
			nn.utils.clip_grad_norm_(classification.parameters(), 5)
			c_optimizer.step()
			c_optimizer.zero_grad()

		max_vali_f1 = evaluate(dataCenter, ds, graphSage, classification, device, max_vali_f1, name, epoch)
	return classification, max_vali_f1

In [74]:
if args.learn_method == 'sup':
	print('GraphSage with Supervised Learning')
elif args.learn_method == 'plus_unsup':
	print('GraphSage with Supervised Learning plus Net Unsupervised Learning')
else:
	print('GraphSage with Net Unsupervised Learning')

for epoch in range(args.epochs):
	print('----------------------EPOCH %d-----------------------' % epoch)
	graphSage, classification = apply_model(dataCenter, ds, graphSage, classification, unsupervised_loss, args.b_sz,
											args.unsup_loss, device, args.learn_method)
	if (epoch + 1) % 2 == 0 and args.learn_method == 'unsup':
		classification, args.max_vali_f1 = train_classification(dataCenter, graphSage, classification, ds, device,
																args.max_vali_f1, args.name)
	if args.learn_method != 'unsup':
		args.max_vali_f1 = evaluate(dataCenter, ds, graphSage, classification, device, args.max_vali_f1, args.name,
									epoch)

GraphSage with Supervised Learning
----------------------EPOCH 0-----------------------
Step [1/68], Loss: 1.9466, Dealed Nodes [1024/1355] 
Step [2/68], Loss: 1.8530, Dealed Nodes [1239/1355] 


C:\Users\Wei Zhou\AppData\Local\Temp\ipykernel_19896\1526409670.py:130: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  neg_samples = random.sample(far_nodes, num_neg) if num_neg < len(far_nodes) else far_nodes
C:\Users\Wei Zhou\AppData\Local\Temp\ipykernel_19896\127083533.py:84: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  samp_neighs = [_set(_sample(to_neigh, num_sample)) if len(to_neigh) >= num_sample else to_neigh for to_neigh in to_neighs]


Step [3/68], Loss: 1.7871, Dealed Nodes [1313/1355] 
Step [4/68], Loss: 1.7182, Dealed Nodes [1337/1355] 
Step [5/68], Loss: 1.6336, Dealed Nodes [1346/1355] 
Step [6/68], Loss: 1.5432, Dealed Nodes [1351/1355] 
Step [7/68], Loss: 1.4436, Dealed Nodes [1353/1355] 
Step [8/68], Loss: 1.3163, Dealed Nodes [1355/1355] 
Step [9/68], Loss: 1.1892, Dealed Nodes [1355/1355] 
Step [10/68], Loss: 1.0488, Dealed Nodes [1355/1355] 
Step [11/68], Loss: 0.9368, Dealed Nodes [1355/1355] 
Step [12/68], Loss: 0.9045, Dealed Nodes [1355/1355] 
Step [13/68], Loss: 1.1622, Dealed Nodes [1355/1355] 
Step [14/68], Loss: 1.7791, Dealed Nodes [1355/1355] 
Step [15/68], Loss: 1.6023, Dealed Nodes [1355/1355] 
Step [16/68], Loss: 1.2177, Dealed Nodes [1355/1355] 
Step [17/68], Loss: 1.0157, Dealed Nodes [1355/1355] 
Step [18/68], Loss: 0.8719, Dealed Nodes [1355/1355] 
Step [19/68], Loss: 0.7181, Dealed Nodes [1355/1355] 
Step [20/68], Loss: 0.6229, Dealed Nodes [1355/1355] 
Step [21/68], Loss: 0.5316, Dealed 

C:\Users\Wei Zhou\AppData\Local\Temp\ipykernel_19896\127083533.py:84: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  samp_neighs = [_set(_sample(to_neigh, num_sample)) if len(to_neigh) >= num_sample else to_neigh for to_neigh in to_neighs]
C:\Users\Wei Zhou\AppData\Local\Temp\ipykernel_19896\1526409670.py:130: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  neg_samples = random.sample(far_nodes, num_neg) if num_neg < len(far_nodes) else far_nodes
C:\Users\Wei Zhou\AppData\Local\Temp\ipykernel_19896\127083533.py:84: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  samp_neighs = [_set(_sample(to_neigh, num_sample)) if len(to_neigh) >= num_sample else to_neigh for to_neigh in to_neighs]


----------------------EPOCH 1-----------------------
Step [1/68], Loss: 0.0386, Dealed Nodes [1037/1355] 
Step [2/68], Loss: 0.0407, Dealed Nodes [1253/1355] 
Step [3/68], Loss: 0.0362, Dealed Nodes [1313/1355] 
Step [4/68], Loss: 0.0355, Dealed Nodes [1341/1355] 
Step [5/68], Loss: 0.0365, Dealed Nodes [1350/1355] 
Step [6/68], Loss: 0.0352, Dealed Nodes [1351/1355] 
Step [7/68], Loss: 0.0288, Dealed Nodes [1354/1355] 
Step [8/68], Loss: 0.0326, Dealed Nodes [1354/1355] 
Step [9/68], Loss: 0.0300, Dealed Nodes [1354/1355] 
Step [10/68], Loss: 0.0310, Dealed Nodes [1355/1355] 
Step [11/68], Loss: 0.0276, Dealed Nodes [1355/1355] 
Step [12/68], Loss: 0.0271, Dealed Nodes [1355/1355] 
Step [13/68], Loss: 0.0257, Dealed Nodes [1355/1355] 
Step [14/68], Loss: 0.0262, Dealed Nodes [1355/1355] 
Step [15/68], Loss: 0.0269, Dealed Nodes [1355/1355] 
Step [16/68], Loss: 0.0251, Dealed Nodes [1355/1355] 
Step [17/68], Loss: 0.0257, Dealed Nodes [1355/1355] 
Step [18/68], Loss: 0.0228, Dealed Nod

...

Step [3/68], Loss: 0.0001, Dealed Nodes [1314/1355] 
Step [4/68], Loss: 0.0001, Dealed Nodes [1343/1355] 
Step [5/68], Loss: 0.0001, Dealed Nodes [1351/1355] 
Step [6/68], Loss: 0.0001, Dealed Nodes [1352/1355] 
Step [7/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [8/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [9/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [10/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [11/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [12/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [13/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [14/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [15/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [16/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [17/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [18/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [19/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [20/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [21/68], Loss: 0.0001, Dealed 

C:\Users\Wei Zhou\AppData\Local\Temp\ipykernel_19896\1526409670.py:130: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  neg_samples = random.sample(far_nodes, num_neg) if num_neg < len(far_nodes) else far_nodes
C:\Users\Wei Zhou\AppData\Local\Temp\ipykernel_19896\127083533.py:84: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  samp_neighs = [_set(_sample(to_neigh, num_sample)) if len(to_neigh) >= num_sample else to_neigh for to_neigh in to_neighs]


Step [3/68], Loss: 0.0001, Dealed Nodes [1323/1355] 
Step [4/68], Loss: 0.0001, Dealed Nodes [1338/1355] 
Step [5/68], Loss: 0.0001, Dealed Nodes [1346/1355] 
Step [6/68], Loss: 0.0001, Dealed Nodes [1353/1355] 
Step [7/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [8/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [9/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [10/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [11/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [12/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [13/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [14/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [15/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [16/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [17/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [18/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [19/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [20/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [21/68], Loss: 0.0001, Dealed 

C:\Users\Wei Zhou\AppData\Local\Temp\ipykernel_19896\1526409670.py:130: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  neg_samples = random.sample(far_nodes, num_neg) if num_neg < len(far_nodes) else far_nodes
C:\Users\Wei Zhou\AppData\Local\Temp\ipykernel_19896\127083533.py:84: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  samp_neighs = [_set(_sample(to_neigh, num_sample)) if len(to_neigh) >= num_sample else to_neigh for to_neigh in to_neighs]


Step [1/68], Loss: 0.0001, Dealed Nodes [1022/1355] 
Step [2/68], Loss: 0.0001, Dealed Nodes [1256/1355] 
Step [3/68], Loss: 0.0001, Dealed Nodes [1331/1355] 
Step [4/68], Loss: 0.0001, Dealed Nodes [1345/1355] 
Step [5/68], Loss: 0.0001, Dealed Nodes [1349/1355] 
Step [6/68], Loss: 0.0001, Dealed Nodes [1353/1355] 
Step [7/68], Loss: 0.0001, Dealed Nodes [1353/1355] 
Step [8/68], Loss: 0.0001, Dealed Nodes [1354/1355] 
Step [9/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [10/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [11/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [12/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [13/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [14/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [15/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [16/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [17/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [18/68], Loss: 0.0001, Dealed Nodes [1355/1355] 
Step [19/68], Loss: 0.0001, Dealed No